In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.neighbors import NearestNeighbors

# Finding counterfactuals

In [90]:
num = 100000
Z = np.random.normal(0, 1, num)
Z_prob = 1 / (1 + np.exp(Z))
X = np.random.binomial(1, Z_prob)
Y = 2 * X + 3 * X * Z + Z + np.random.normal(0, 1, num)
df = pd.DataFrame({"X": X, "Z": Z, "Y": Y, "X_Z": X * Z})

In [91]:
results = sm.OLS(Y, sm.add_constant(df.drop(columns=["Y"]))).fit()
results.params

const    0.008310
X        1.994919
Z        1.005752
X_Z      2.997433
dtype: float64

In [92]:
treated = df[df.X == 1]
untreated = df[df.X == 0]
untreated.Y.mean()

0.42062654966051427

In [93]:
# ATT
treated_cf_Y = results.predict(pd.DataFrame({"const": 1, "X": 0, "Z": treated.Z, "X_Z": 0}))
treated.Y.mean() - treated_cf_Y.mean()

0.7353446111946642

In [96]:
2 + 3 * treated.Z.mean()

0.7393470059471832

In [94]:
#ATUT
untreated_cf_Y = results.predict(pd.DataFrame({"const": 1, "X": 1, "Z": untreated.Z, "X_Z": untreated.Z}))
untreated_cf_Y.mean() - untreated.Y.mean()

3.223741673634595

In [97]:
2 + 3 * untreated.Z.mean()

3.2298750446064006

In [95]:
#ATE
((treated.Y.mean() - treated_cf_Y.mean()) * treated.shape[0] + (untreated_cf_Y.mean() - untreated.Y.mean()) * untreated.shape[0]) \
/ df.shape[0]

1.9844701685982609

In [98]:
2 + 3 * df.Z.mean()

1.9895422707933372

In [55]:
num = 100000
Z_mean = 0
Z = np.random.normal(Z_mean, 1, num)
Z_prob = 1 / (1 + np.exp(Z))
X = np.random.binomial(1, Z_prob)
Y = 2 * X + np.random.normal(0, 1, num)
df = pd.DataFrame({"X": X, "Y": Y, "Z": Z})

### Linear regression for prediction: without X * Z

In [56]:
model = sm.OLS(Y, sm.add_constant(df[["X", "Z"]]))
results = model.fit()
results.params

const    0.005799
X        1.991686
Z       -0.000920
dtype: float64

In [57]:
treated = df[df.X == 1]
untreated = df[df.X == 0]

treated_cf = sm.add_constant(pd.DataFrame({"X": 0, "Z": treated["Z"]}), has_constant = "add")
treated_cf_prediction = results.predict(treated_cf)
untreated_cf = sm.add_constant(pd.DataFrame({"X": 1, "Z": untreated["Z"]}), has_constant = "add")
untreated_cf_prediction = results.predict(untreated_cf)

print("ATT effect:", treated.Y.mean() - treated_cf_prediction.mean())
print("ATUT effect:", untreated_cf_prediction.mean() - untreated.Y.mean())
print("ATE effect:", ((treated.Y.mean() - treated_cf_prediction.mean()) * treated.shape[0] + (untreated_cf_prediction.mean() - untreated.Y.mean()) * untreated.shape[0]) / (untreated.shape[0] + treated.shape[0]))

ATT effect: 1.9916860517242791
ATUT effect: 1.9916860517241906
ATE effect: 1.9916860517242347


In [58]:
num = 100000
Z_mean = 0.5
Z = np.random.normal(Z_mean, 1, num)
Z_prob = 1 / (1 + np.exp(Z))
X = np.random.binomial(1, Z_prob)
Y = 2 * X + 3 * X * Z + np.random.normal(0, 1, num)
df = pd.DataFrame({"X": X, "Y": Y, "Z": Z, "X_Z": X * Z})

### Linear regression for prediction: with X * Z

In [59]:
model = sm.OLS(Y, sm.add_constant(df[["X", "Z", "X_Z"]]))
results = model.fit()
results.params

const    0.005096
X        1.992708
Z       -0.000339
X_Z      2.992404
dtype: float64

In [60]:
treated = df[df.X == 1]
untreated = df[df.X == 0]

treated_cf = sm.add_constant(pd.DataFrame({"X": 0, "Z": treated["Z"], "X_Z": 0}), has_constant = "add")
treated_cf_prediction = results.predict(treated_cf)
untreated_cf = sm.add_constant(pd.DataFrame({"X": 1, "Z": untreated["Z"], "X_Z": untreated["Z"]}), has_constant = "add")
untreated_cf_prediction = results.predict(untreated_cf)

print("treated Z mean:", treated.Z.mean(), "2 + 3 * Z = ", 2 + 3 * treated.Z.mean())
print("ATT effect:", treated.Y.mean() - treated_cf_prediction.mean())
print("untreated Z mean:", untreated.Z.mean(), "2 + 3 * Z = ", 2 + 3 * untreated.Z.mean())
print("ATUT effect:", untreated_cf_prediction.mean() - untreated.Y.mean())
print("overall Z mean:", df.Z.mean(), "2 + 3 * Z = ", 2 + 3 * df.Z.mean())
print("ATE effect:", ((treated.Y.mean() - treated_cf_prediction.mean()) * treated.shape[0] + (untreated_cf_prediction.mean() - untreated.Y.mean()) * untreated.shape[0]) / (untreated.shape[0] + treated.shape[0]))

treated Z mean: -0.004156587346241618 2 + 3 * Z =  1.9875302379612751
ATT effect: 1.9802695641301544
untreated Z mean: 0.8297234233615438 2 + 3 * Z =  4.489170270084632
ATUT effect: 4.475575858346705
overall Z mean: 0.4985479771089467 2 + 3 * Z =  3.49564393132684
ATE effect: 3.4845649635986016


### If Z_mean is changed, this unweighted ATE effect will deviate from the correct ATE effect

In [32]:
# Note that if you don't weight by the number of items in treated and untreated, you get the wrong result
print("ATE effect (wrong):", ((treated.Y.mean() - treated_cf_prediction.mean()) + (untreated_cf_prediction.mean() - untreated.Y.mean())) / 2)

ATE effect (wrong): 3.2321026886315014


### Optimal Treatment Effect (In the homework I called this Marginal, but it's not quite the right term)

In [57]:
arg = (untreated_cf_prediction.reset_index()[0] - untreated.reset_index().Y).argmax()
print(untreated_cf_prediction.iloc[arg])
print(untreated.iloc[arg])

12.65053835434058
X      0.000000
Y     -1.766167
Z      3.550861
X_Z    0.000000
Name: 89227, dtype: float64


### Matching for prediction

In [104]:
neighbors = NearestNeighbors(n_neighbors = 1).fit(untreated[["Z"]])
distances, indices = neighbors.kneighbors(treated[["Z"]])
treated.Y.mean() - untreated.iloc[indices.ravel()].Y.mean()

0.7246706959896432

In [105]:
neighbors = NearestNeighbors(n_neighbors = 1).fit(treated[["Z"]])
distances, indices = neighbors.kneighbors(untreated[["Z"]])
treated.iloc[indices.ravel()].Y.mean() - untreated.Y.mean()

3.2332933716555745

In [61]:
neighbors = NearestNeighbors(n_neighbors = 1).fit(untreated[["Z"]])
distances, indices = neighbors.kneighbors(treated[["Z"]])
treated.Y.mean() - untreated.iloc[indices.ravel()].Y.mean()

1.9655741712841313

In [62]:
neighbors = NearestNeighbors(n_neighbors = 1).fit(untreated[["Z"]])
distances, indices = neighbors.kneighbors(treated[["Z"]])
treated_cf = untreated.iloc[indices.ravel()]

In [63]:
neighbors = NearestNeighbors(n_neighbors = 1).fit(treated[["Z"]])
distances, indices = neighbors.kneighbors(untreated[["Z"]])
untreated_cf = treated.iloc[indices.ravel()]

In [64]:
print("ATT effect:", treated.Y.mean() - treated_cf.Y.mean())
print("ATUT effect:", untreated_cf.Y.mean() - untreated.Y.mean())
print("ATE effect:", ((treated.Y.mean() - treated_cf.Y.mean()) * treated.shape[0] + (untreated_cf.Y.mean() - untreated.Y.mean()) * untreated.shape[0]) / (untreated.shape[0] + treated.shape[0]))

ATT effect: 1.9655741712841313
ATUT effect: 4.457401191013728
ATE effect: 3.467772090128119


### Optimal Treatment Effect

In [50]:
arg = (untreated_cf.reset_index().Y - untreated.reset_index().Y).argmax()
print(untreated_cf.iloc[arg])
print(untreated.iloc[arg])

X       1.000000
Y      13.336748
Z       3.568249
X_Z     3.568249
Name: 3451, dtype: float64
X      0.000000
Y     -1.766167
Z      3.550861
X_Z    0.000000
Name: 89227, dtype: float64
